In [1]:
import os
import numpy as np

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets

In [2]:
lr = 1e-3
batch_size =64
num_epoch = 10

In [3]:
ckpt_dir = "./cehckpoint"
log_dir   = "./log"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels=10, kernel_size=5, stride=1, padding=0, bias=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(in_channels = 10, out_channels=20, kernel_size=5, stride=1, padding=0, bias=True)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = nn.ReLU()
        
        self.fc1 = nn.Linear(in_features = 320,  out_features=50, bias=True)
        self.relu1_fc1 = nn.ReLU()
        self.drop1_fc1 = nn.Dropout2d(p=0.5)
        
        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)
    
    def forward(self,x):
        
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.relu2(x)
        
        x = x.view(-1, 320)
        
        x = self.fc1(x)
        x = self.relu1_fc1(x)
        x = self.drop1_fc1(x)
        
        x = self.fc2(x)
        
        return x

In [6]:
def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.mkdir(ckpt_dir)
        
        torch.save({"net":net.state_dict(), "optim":optim.state_dict()}, f"./{ckpt_dir}/epoch{epoch}.pth")

In [7]:
def load(ckpt_dir, net, optim):
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort()
    
    dict_model = torch.load(f"./{ckpt_dir}/{ckpt_lst[-1]}")
    
    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    
    return net, optim

In [8]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])
dataset = datasets.MNIST(download=True, root="./", train=True, transform=transform)

In [9]:
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [10]:
num_data = len(loader.dataset)
num_batch = int(np.ceil(num_data/batch_size))

In [11]:
net = Net().to(device)
params = net.parameters()

fn_loss  = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output: torch.softmax(output, dim=1)
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()

optim = torch.optim.Adam(params, lr=lr)

writer = SummaryWriter(log_dir=log_dir)

In [12]:
for epoch in range(1, num_epoch+1):
    net.train()
    
    loss_arr = []
    acc_arr  = []
    
    for batch, (inputs,label) in enumerate(loader, 1):
        inputs = inputs.to(device)
        label  = label.to(device)
        output = net(inputs)
        pred   = fn_pred(output)
        
        optim.zero_grad()
        loss= fn_loss(output, label)
        acc = fn_acc(pred, label)
        loss.backward()
        
        optim.step()
        
        
        loss_arr += [loss.item()]
        acc_arr  += [acc.item()]
        
        print(f"Train : Epoch {epoch:04d} | BATCH {batch:04d} / {num_batch:04d}| LOSS {np.mean(loss_arr):.4f} ACC {np.mean(acc_arr):.4f}")
    
    writer.add_scalar("loss", np.mean(loss_arr), epoch)
    writer.add_scalar("acc", np.mean(acc_arr), epoch)
    
    save(ckpt_dir = ckpt_dir, net=net, optim=optim, epoch=epoch)
writer.close()

Train : Epoch 0001 | BATCH 0010 / 0001| LOSS 2.3169 ACC 0.0469
Train : Epoch 0001 | BATCH 0010 / 0002| LOSS 2.3201 ACC 0.0703
Train : Epoch 0001 | BATCH 0010 / 0003| LOSS 2.3142 ACC 0.0938
Train : Epoch 0001 | BATCH 0010 / 0004| LOSS 2.3139 ACC 0.0859
Train : Epoch 0001 | BATCH 0010 / 0005| LOSS 2.3125 ACC 0.0844
Train : Epoch 0001 | BATCH 0010 / 0006| LOSS 2.3113 ACC 0.0911
Train : Epoch 0001 | BATCH 0010 / 0007| LOSS 2.3089 ACC 0.0982
Train : Epoch 0001 | BATCH 0010 / 0008| LOSS 2.3046 ACC 0.1133
Train : Epoch 0001 | BATCH 0010 / 0009| LOSS 2.3026 ACC 0.1181
Train : Epoch 0001 | BATCH 0010 / 0010| LOSS 2.2986 ACC 0.1266
Train : Epoch 0001 | BATCH 0010 / 0011| LOSS 2.2969 ACC 0.1321
Train : Epoch 0001 | BATCH 0010 / 0012| LOSS 2.2939 ACC 0.1432
Train : Epoch 0001 | BATCH 0010 / 0013| LOSS 2.2919 ACC 0.1430
Train : Epoch 0001 | BATCH 0010 / 0014| LOSS 2.2896 ACC 0.1406
Train : Epoch 0001 | BATCH 0010 / 0015| LOSS 2.2849 ACC 0.1458
Train : Epoch 0001 | BATCH 0010 / 0016| LOSS 2.2827 ACC

Train : Epoch 0001 | BATCH 0010 / 0143| LOSS 1.3463 ACC 0.5461
Train : Epoch 0001 | BATCH 0010 / 0144| LOSS 1.3414 ACC 0.5484
Train : Epoch 0001 | BATCH 0010 / 0145| LOSS 1.3370 ACC 0.5501
Train : Epoch 0001 | BATCH 0010 / 0146| LOSS 1.3321 ACC 0.5518
Train : Epoch 0001 | BATCH 0010 / 0147| LOSS 1.3274 ACC 0.5535
Train : Epoch 0001 | BATCH 0010 / 0148| LOSS 1.3219 ACC 0.5553
Train : Epoch 0001 | BATCH 0010 / 0149| LOSS 1.3175 ACC 0.5572
Train : Epoch 0001 | BATCH 0010 / 0150| LOSS 1.3142 ACC 0.5584
Train : Epoch 0001 | BATCH 0010 / 0151| LOSS 1.3106 ACC 0.5597
Train : Epoch 0001 | BATCH 0010 / 0152| LOSS 1.3054 ACC 0.5619
Train : Epoch 0001 | BATCH 0010 / 0153| LOSS 1.3015 ACC 0.5632
Train : Epoch 0001 | BATCH 0010 / 0154| LOSS 1.2970 ACC 0.5647
Train : Epoch 0001 | BATCH 0010 / 0155| LOSS 1.2924 ACC 0.5660
Train : Epoch 0001 | BATCH 0010 / 0156| LOSS 1.2884 ACC 0.5672
Train : Epoch 0001 | BATCH 0010 / 0157| LOSS 1.2856 ACC 0.5686
Train : Epoch 0001 | BATCH 0010 / 0158| LOSS 1.2823 ACC

Train : Epoch 0001 | BATCH 0010 / 0287| LOSS 0.9390 ACC 0.6898
Train : Epoch 0001 | BATCH 0010 / 0288| LOSS 0.9377 ACC 0.6904
Train : Epoch 0001 | BATCH 0010 / 0289| LOSS 0.9354 ACC 0.6912
Train : Epoch 0001 | BATCH 0010 / 0290| LOSS 0.9345 ACC 0.6918
Train : Epoch 0001 | BATCH 0010 / 0291| LOSS 0.9327 ACC 0.6925
Train : Epoch 0001 | BATCH 0010 / 0292| LOSS 0.9320 ACC 0.6929
Train : Epoch 0001 | BATCH 0010 / 0293| LOSS 0.9303 ACC 0.6935
Train : Epoch 0001 | BATCH 0010 / 0294| LOSS 0.9287 ACC 0.6939
Train : Epoch 0001 | BATCH 0010 / 0295| LOSS 0.9273 ACC 0.6945
Train : Epoch 0001 | BATCH 0010 / 0296| LOSS 0.9263 ACC 0.6947
Train : Epoch 0001 | BATCH 0010 / 0297| LOSS 0.9252 ACC 0.6952
Train : Epoch 0001 | BATCH 0010 / 0298| LOSS 0.9235 ACC 0.6960
Train : Epoch 0001 | BATCH 0010 / 0299| LOSS 0.9214 ACC 0.6966
Train : Epoch 0001 | BATCH 0010 / 0300| LOSS 0.9203 ACC 0.6970
Train : Epoch 0001 | BATCH 0010 / 0301| LOSS 0.9195 ACC 0.6972
Train : Epoch 0001 | BATCH 0010 / 0302| LOSS 0.9174 ACC

Train : Epoch 0001 | BATCH 0010 / 0431| LOSS 0.7628 ACC 0.7525
Train : Epoch 0001 | BATCH 0010 / 0432| LOSS 0.7615 ACC 0.7530
Train : Epoch 0001 | BATCH 0010 / 0433| LOSS 0.7609 ACC 0.7534
Train : Epoch 0001 | BATCH 0010 / 0434| LOSS 0.7602 ACC 0.7537
Train : Epoch 0001 | BATCH 0010 / 0435| LOSS 0.7609 ACC 0.7536
Train : Epoch 0001 | BATCH 0010 / 0436| LOSS 0.7599 ACC 0.7539
Train : Epoch 0001 | BATCH 0010 / 0437| LOSS 0.7590 ACC 0.7543
Train : Epoch 0001 | BATCH 0010 / 0438| LOSS 0.7581 ACC 0.7546
Train : Epoch 0001 | BATCH 0010 / 0439| LOSS 0.7575 ACC 0.7548
Train : Epoch 0001 | BATCH 0010 / 0440| LOSS 0.7562 ACC 0.7552
Train : Epoch 0001 | BATCH 0010 / 0441| LOSS 0.7553 ACC 0.7555
Train : Epoch 0001 | BATCH 0010 / 0442| LOSS 0.7540 ACC 0.7559
Train : Epoch 0001 | BATCH 0010 / 0443| LOSS 0.7531 ACC 0.7562
Train : Epoch 0001 | BATCH 0010 / 0444| LOSS 0.7522 ACC 0.7565
Train : Epoch 0001 | BATCH 0010 / 0445| LOSS 0.7516 ACC 0.7568
Train : Epoch 0001 | BATCH 0010 / 0446| LOSS 0.7507 ACC

Train : Epoch 0001 | BATCH 0010 / 0576| LOSS 0.6597 ACC 0.7880
Train : Epoch 0001 | BATCH 0010 / 0577| LOSS 0.6594 ACC 0.7882
Train : Epoch 0001 | BATCH 0010 / 0578| LOSS 0.6588 ACC 0.7884
Train : Epoch 0001 | BATCH 0010 / 0579| LOSS 0.6585 ACC 0.7886
Train : Epoch 0001 | BATCH 0010 / 0580| LOSS 0.6579 ACC 0.7887
Train : Epoch 0001 | BATCH 0010 / 0581| LOSS 0.6579 ACC 0.7888
Train : Epoch 0001 | BATCH 0010 / 0582| LOSS 0.6574 ACC 0.7890
Train : Epoch 0001 | BATCH 0010 / 0583| LOSS 0.6567 ACC 0.7892
Train : Epoch 0001 | BATCH 0010 / 0584| LOSS 0.6562 ACC 0.7894
Train : Epoch 0001 | BATCH 0010 / 0585| LOSS 0.6555 ACC 0.7895
Train : Epoch 0001 | BATCH 0010 / 0586| LOSS 0.6546 ACC 0.7899
Train : Epoch 0001 | BATCH 0010 / 0587| LOSS 0.6538 ACC 0.7901
Train : Epoch 0001 | BATCH 0010 / 0588| LOSS 0.6533 ACC 0.7903
Train : Epoch 0001 | BATCH 0010 / 0589| LOSS 0.6530 ACC 0.7904
Train : Epoch 0001 | BATCH 0010 / 0590| LOSS 0.6525 ACC 0.7906
Train : Epoch 0001 | BATCH 0010 / 0591| LOSS 0.6519 ACC

...

Train : Epoch 0010 | BATCH 0010 / 0247| LOSS 0.1190 ACC 0.9646
Train : Epoch 0010 | BATCH 0010 / 0248| LOSS 0.1190 ACC 0.9647
Train : Epoch 0010 | BATCH 0010 / 0249| LOSS 0.1189 ACC 0.9648
Train : Epoch 0010 | BATCH 0010 / 0250| LOSS 0.1192 ACC 0.9647
Train : Epoch 0010 | BATCH 0010 / 0251| LOSS 0.1194 ACC 0.9646
Train : Epoch 0010 | BATCH 0010 / 0252| LOSS 0.1198 ACC 0.9646
Train : Epoch 0010 | BATCH 0010 / 0253| LOSS 0.1197 ACC 0.9646
Train : Epoch 0010 | BATCH 0010 / 0254| LOSS 0.1198 ACC 0.9644
Train : Epoch 0010 | BATCH 0010 / 0255| LOSS 0.1199 ACC 0.9643
Train : Epoch 0010 | BATCH 0010 / 0256| LOSS 0.1199 ACC 0.9643
Train : Epoch 0010 | BATCH 0010 / 0257| LOSS 0.1202 ACC 0.9643
Train : Epoch 0010 | BATCH 0010 / 0258| LOSS 0.1199 ACC 0.9643
Train : Epoch 0010 | BATCH 0010 / 0259| LOSS 0.1204 ACC 0.9642
Train : Epoch 0010 | BATCH 0010 / 0260| LOSS 0.1201 ACC 0.9643
Train : Epoch 0010 | BATCH 0010 / 0261| LOSS 0.1199 ACC 0.9644
Train : Epoch 0010 | BATCH 0010 / 0262| LOSS 0.1198 ACC

Train : Epoch 0010 | BATCH 0010 / 0391| LOSS 0.1278 ACC 0.9620
Train : Epoch 0010 | BATCH 0010 / 0392| LOSS 0.1278 ACC 0.9620
Train : Epoch 0010 | BATCH 0010 / 0393| LOSS 0.1278 ACC 0.9620
Train : Epoch 0010 | BATCH 0010 / 0394| LOSS 0.1279 ACC 0.9620
Train : Epoch 0010 | BATCH 0010 / 0395| LOSS 0.1280 ACC 0.9619
Train : Epoch 0010 | BATCH 0010 / 0396| LOSS 0.1279 ACC 0.9620
Train : Epoch 0010 | BATCH 0010 / 0397| LOSS 0.1281 ACC 0.9620
Train : Epoch 0010 | BATCH 0010 / 0398| LOSS 0.1281 ACC 0.9620
Train : Epoch 0010 | BATCH 0010 / 0399| LOSS 0.1280 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0400| LOSS 0.1280 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0401| LOSS 0.1280 ACC 0.9622
Train : Epoch 0010 | BATCH 0010 / 0402| LOSS 0.1280 ACC 0.9622
Train : Epoch 0010 | BATCH 0010 / 0403| LOSS 0.1278 ACC 0.9622
Train : Epoch 0010 | BATCH 0010 / 0404| LOSS 0.1278 ACC 0.9622
Train : Epoch 0010 | BATCH 0010 / 0405| LOSS 0.1285 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0406| LOSS 0.1283 ACC

Train : Epoch 0010 | BATCH 0010 / 0536| LOSS 0.1289 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0537| LOSS 0.1288 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0538| LOSS 0.1288 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0539| LOSS 0.1290 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0540| LOSS 0.1289 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0541| LOSS 0.1288 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0542| LOSS 0.1292 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0543| LOSS 0.1291 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0544| LOSS 0.1290 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0545| LOSS 0.1290 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0546| LOSS 0.1290 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0547| LOSS 0.1288 ACC 0.9622
Train : Epoch 0010 | BATCH 0010 / 0548| LOSS 0.1289 ACC 0.9622
Train : Epoch 0010 | BATCH 0010 / 0549| LOSS 0.1290 ACC 0.9621
Train : Epoch 0010 | BATCH 0010 / 0550| LOSS 0.1289 ACC 0.9622
Train : Epoch 0010 | BATCH 0010 / 0551| LOSS 0.1289 ACC

Train : Epoch 0010 | BATCH 0010 / 0681| LOSS 0.1304 ACC 0.9614
Train : Epoch 0010 | BATCH 0010 / 0682| LOSS 0.1304 ACC 0.9614
Train : Epoch 0010 | BATCH 0010 / 0683| LOSS 0.1303 ACC 0.9614
Train : Epoch 0010 | BATCH 0010 / 0684| LOSS 0.1301 ACC 0.9614
Train : Epoch 0010 | BATCH 0010 / 0685| LOSS 0.1300 ACC 0.9614
Train : Epoch 0010 | BATCH 0010 / 0686| LOSS 0.1300 ACC 0.9615
Train : Epoch 0010 | BATCH 0010 / 0687| LOSS 0.1299 ACC 0.9615
Train : Epoch 0010 | BATCH 0010 / 0688| LOSS 0.1297 ACC 0.9615
Train : Epoch 0010 | BATCH 0010 / 0689| LOSS 0.1298 ACC 0.9615
Train : Epoch 0010 | BATCH 0010 / 0690| LOSS 0.1300 ACC 0.9615
Train : Epoch 0010 | BATCH 0010 / 0691| LOSS 0.1301 ACC 0.9614
Train : Epoch 0010 | BATCH 0010 / 0692| LOSS 0.1301 ACC 0.9615
Train : Epoch 0010 | BATCH 0010 / 0693| LOSS 0.1300 ACC 0.9615
Train : Epoch 0010 | BATCH 0010 / 0694| LOSS 0.1301 ACC 0.9614
Train : Epoch 0010 | BATCH 0010 / 0695| LOSS 0.1301 ACC 0.9615
Train : Epoch 0010 | BATCH 0010 / 0696| LOSS 0.1303 ACC

Train : Epoch 0010 | BATCH 0010 / 0826| LOSS 0.1312 ACC 0.9611
Train : Epoch 0010 | BATCH 0010 / 0827| LOSS 0.1312 ACC 0.9611
Train : Epoch 0010 | BATCH 0010 / 0828| LOSS 0.1313 ACC 0.9610
Train : Epoch 0010 | BATCH 0010 / 0829| LOSS 0.1312 ACC 0.9611
Train : Epoch 0010 | BATCH 0010 / 0830| LOSS 0.1312 ACC 0.9611
Train : Epoch 0010 | BATCH 0010 / 0831| LOSS 0.1311 ACC 0.9611
Train : Epoch 0010 | BATCH 0010 / 0832| LOSS 0.1311 ACC 0.9611
Train : Epoch 0010 | BATCH 0010 / 0833| LOSS 0.1310 ACC 0.9611
Train : Epoch 0010 | BATCH 0010 / 0834| LOSS 0.1309 ACC 0.9612
Train : Epoch 0010 | BATCH 0010 / 0835| LOSS 0.1310 ACC 0.9611
Train : Epoch 0010 | BATCH 0010 / 0836| LOSS 0.1311 ACC 0.9611
Train : Epoch 0010 | BATCH 0010 / 0837| LOSS 0.1309 ACC 0.9612
Train : Epoch 0010 | BATCH 0010 / 0838| LOSS 0.1309 ACC 0.9612
Train : Epoch 0010 | BATCH 0010 / 0839| LOSS 0.1309 ACC 0.9612
Train : Epoch 0010 | BATCH 0010 / 0840| LOSS 0.1309 ACC 0.9612
Train : Epoch 0010 | BATCH 0010 / 0841| LOSS 0.1309 ACC